In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np

In [ ]:
import nltk
# nltk.download('stopwords') #run once 
from nltk.corpus import stopwords 

In [2]:
import re

In [ ]:
query = "adani"

tweets = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):   
    if i>20000:
        break

    else:
        # extract specific tweets
        tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])

df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
df.to_csv('sentiment.csv', mode = 'a')

In [ ]:
eng_stop_words = list(stopwords.words('english'))

In [14]:
# regex

def filtering(text):
    text = text.lower()
    
    # @ aur # frq ke liye imp 
    # remove stop words
    #polarity

filtering("yash @yash22")


yash   


In [ ]:
# election oriented

# like search 2024 elections trump biden .....

# removing emotionless words

# freq of top words
# segregate according to state and freq of top words

# overall polarity
# segregate according to state and polarity

# election advertising